In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier  # Changed to LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import sem

In [ ]:
# Define seeds and flip percentages
seeds = [44, 440, 200, 1000, 300, 430, 5490, 234, 456, 564]
flip_percentages = [i/100 for i in range(0, 71, 5)]
metric_names = ['Accuracy', 'Selection rate', 'TPR', 'TNR', 'FPR', 'FNR', 'PPV', 'NPV', 'ROC AUC', 'F1']
fairness_metrics = ['Independence (M_ind)', 'Separation (M_sep)', 'Sufficiency (M_suff)']

# Define experiments
experiments = [
    {'train': 'true', 'test': 'true', 'name': 'True-True'},
    {'train': 'true', 'test': 'bias', 'name': 'True-Bias'},
    {'train': 'bias', 'test': 'true', 'name': 'Bias-True'},
    {'train': 'bias', 'test': 'bias', 'name': 'Bias-Bias'}
]

In [ ]:
# Initialize results dictionary
all_experiment_results = {
    exp['name']: {
        'metrics_before': [[] for _ in seeds],
        'metrics_after': [[] for _ in seeds],
        'fairness_before': [[] for _ in seeds],
        'fairness_after': [[] for _ in seeds],
        'bias_diagnosis': [[] for _ in seeds],
        'feature_importance': [[] for _ in seeds]  # Added for feature importance storage
    } for exp in experiments
}

# Function to compute metrics
def compute_metrics(y_true, y_pred, group_name='Overall'):
    acc = accuracy_score(y_true, y_pred)
    sel_rate = np.mean(y_pred)
    tpr = recall_score(y_true, y_pred, zero_division=0)
    ppv = precision_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
    fpr = fp / (tn + fp) if (tn + fp) > 0 else 0
    fnr = fn / (tp + fn) if (tp + fn) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    return {
        'Group': group_name,
        'Accuracy': round(acc, 4),
        'Selection rate': round(sel_rate, 4),
        'TPR': round(tpr, 4),
        'TNR': round(tnr, 4),
        'FPR': round(fpr, 4),
        'FNR': round(fnr, 4),
        'PPV': round(ppv, 4),
        'NPV': round(npv, 4),
        'ROC AUC': round(roc_auc, 4),
        'F1': round(f1, 4)
    }

In [ ]:

# Function for fairness diagnosis
def fairness_diagnosis(metrics_male, metrics_female):
    groups = ['Male', 'Female']
    metrics_list = [metrics_male, metrics_female]
    sr_diffs = [max(abs(g1['Selection rate'] - g2['Selection rate']) for j, g2 in enumerate(metrics_list) if i != j) for i, g1 in enumerate(metrics_list)]
    m_ind = max(sr_diffs)
    sep_scores = [max(abs(g1['TPR'] - g2['TPR']) + abs(g1['FPR'] - g2['FPR']) for j, g2 in enumerate(metrics_list) if i != j) for i, g1 in enumerate(metrics_list)]
    m_sep = max(sep_scores)
    suff_scores = [max(abs(g1['PPV'] - g2['PPV']) + abs(g1['NPV'] - g2['NPV']) for j, g2 in enumerate(metrics_list) if i != j) for i, g1 in enumerate(metrics_list)]
    m_suff = max(suff_scores)
    return pd.DataFrame({
        'Metric': ['Independence (M_ind)', 'Separation (M_sep)', 'Sufficiency (M_suff)'],
        'Value': [round(m_ind, 4), round(m_sep, 4), round(m_suff, 4)]
    })

In [ ]:
# Function for bias diagnosis
def bias_diagnosis(y_true, y_bias, sensitive):
    male_mask = (sensitive == 'Male')
    female_mask = (sensitive == 'Female')
    error_rate_male = np.mean(y_true[male_mask] != y_bias[male_mask]) if male_mask.sum() > 0 else 0
    error_rate_female = np.mean(y_true[female_mask] != y_bias[female_mask]) if female_mask.sum() > 0 else 0
    return {'Male': round(error_rate_male, 4), 'Female': round(error_rate_female, 4)}

In [ ]:
# Function for mitigation
def apply_mitigation(gbm, X_val, y_pred_proba, sensitive, y_val, sensitive_val):
    male_mask_val = (sensitive_val == 'Male')
    female_mask_val = (sensitive_val == 'Female')
    y_pred_proba_val = gbm.predict_proba(X_val)
    best_thresholds = {'Male': 0.5, 'Female': 0.5}
    best_score = float('inf')
    thresholds = np.arange(0.3, 0.8, 0.1)
    for t_male in thresholds:
        for t_female in thresholds:
            y_pred_val = np.zeros_like(y_val)
            y_pred_val[male_mask_val] = (y_pred_proba_val[male_mask_val][:, 1] >= t_male).astype(int)
            y_pred_val[female_mask_val] = (y_pred_proba_val[female_mask_val][:, 1] >= t_female).astype(int)
            male_metrics = compute_metrics(y_val[male_mask_val], y_pred_val[male_mask_val], group_name='Male')
            female_metrics = compute_metrics(y_val[female_mask_val], y_pred_val[female_mask_val], group_name='Female')
            fairness = fairness_diagnosis(male_metrics, female_metrics)
            sep_score = fairness['Value'][1]
            acc = compute_metrics(y_val, y_pred_val)['Accuracy']
            score = sep_score - acc
            if score < best_score:
                best_score = score
                best_thresholds = {'Male': t_male, 'Female': t_female}
    y_pred_mitigated = np.zeros_like(y_pred_proba[:, 1])
    male_mask = (sensitive == 'Male')
    female_mask = (sensitive == 'Female')
    y_pred_mitigated[male_mask] = (y_pred_proba[male_mask][:, 1] >= best_thresholds['Male']).astype(int)
    y_pred_mitigated[female_mask] = (y_pred_proba[female_mask][:, 1] >= best_thresholds['Female']).astype(int)
    return y_pred_mitigated

In [ ]:
# Function to plot metrics across flips (with overall line and mean ± SE annotations)
def plot_metrics_across_flips(metrics_results, flip_percentages, metric_names, title_suffix, file_name):
    plt.figure(figsize=(15, 10))
    n_metrics = len(metric_names)
    n_cols = 4
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    for i, metric in enumerate(metric_names):
        # Calculate mean and SE across seeds
        male_values = np.array([[result['Male'][metric] for result in metrics] for metrics in metrics_results])
        female_values = np.array([[result['Female'][metric] for result in metrics] for metrics in metrics_results])
        overall_values = np.array([[result['Overall'][metric] for result in metrics] for metrics in metrics_results])
        male_mean = np.mean(male_values, axis=0)
        female_mean = np.mean(female_values, axis=0)
        overall_mean = np.mean(overall_values, axis=0)
        male_se = sem(male_values, axis=0)
        female_se = sem(female_values, axis=0)
        overall_se = sem(overall_values, axis=0)
        
        # Calculate overall mean and SE across all flip percentages
        overall_male_mean = np.mean(male_mean)
        overall_male_se = np.mean(male_se)
        overall_female_mean = np.mean(female_mean)
        overall_female_se = np.mean(female_se)
        overall_overall_mean = np.mean(overall_mean)
        overall_overall_se = np.mean(overall_se)
        
        plt.subplot(n_rows, n_cols, i+1)
        plt.plot(flip_percentages, male_mean, 'o-', color='#1f77b4', label='Male Mean')
        plt.fill_between(flip_percentages, male_mean - male_se, male_mean + male_se, color='#1f77b4', alpha=0.2, label='Male Mean ± SE')
        plt.plot(flip_percentages, female_mean, 'o-', color='#ff7f0e', label='Female Mean')
        plt.fill_between(flip_percentages, female_mean - female_se, female_mean + female_se, color='#ff7f0e', alpha=0.2, label='Female Mean ± SE')
        plt.plot(flip_percentages, overall_mean, 'o-', color='#2ca02c', label='Overall Mean')
        plt.fill_between(flip_percentages, overall_mean - overall_se, overall_mean + overall_se, color='#2ca02c', alpha=0.2, label='Overall Mean ± SE')
        
        # Add overall mean ± SE as text annotation
        plt.text(0.02, 0.98, f'Male: {overall_male_mean:.4f} ± {overall_male_se:.4f}\nFemale: {overall_female_mean:.4f} ± {overall_female_se:.4f}\nOverall: {overall_overall_mean:.4f} ± {overall_overall_se:.4f}',
                 transform=plt.gca().transAxes, fontsize=8, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.xlabel('Flip Percentage')
        plt.ylabel(metric)
        plt.title(f'{metric} vs Flip Percentage')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.ylim(0.0, 1.0)  # Set y-axis limits to 0.0 to 1.0
    
    plt.suptitle(f'Metrics vs Flip Percentage ({title_suffix})', fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    with PdfPages(file_name) as pdf:
        pdf.savefig()
    plt.close()

In [ ]:
# Function to plot VIP (Variable Importance) across seeds
def plot_vip_across_seeds(feature_importance_results, flip_percentages, feature_names, exp_name):
    # Calculate mean feature importance across seeds for each flip percentage
    n_flips = len(flip_percentages)
    n_features = len(feature_names)
    
    # Create a 3D array: seeds × flips × features
    all_importance = np.array([[[fi[flip_idx][feature_idx] for feature_idx in range(n_features)] 
                              for flip_idx in range(n_flips)] 
                             for fi in feature_importance_results])
    
    # Calculate mean across seeds
    mean_importance = np.mean(all_importance, axis=0)  # flips × features
    
    # Plot VIP for each flip percentage
    plt.figure(figsize=(15, 20))
    n_cols = 4
    n_rows = (n_flips + n_cols - 1) // n_cols
    
    for flip_idx, flip_pct in enumerate(flip_percentages):
        # Get feature importance for this flip percentage
        importance = mean_importance[flip_idx]
        
        # Create a DataFrame for feature importance
        feature_importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importance
        }).sort_values('Importance', ascending=True)
        
        # Plot VIP
        plt.subplot(n_rows, n_cols, flip_idx+1)
        plt.barh(range(len(feature_importance_df)), feature_importance_df['Importance'], align='center')
        plt.yticks(range(len(feature_importance_df)), feature_importance_df['Feature'], fontsize=8)
        plt.xlabel('Importance')
        plt.title(f'Flip {flip_pct*100}%')
        plt.grid(True, alpha=0.3)
    
    plt.suptitle(f'Mean Variable Importance Across Seeds ({exp_name})', fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    with PdfPages(f"vip_mean_{exp_name}.pdf") as pdf:
        pdf.savefig()
    plt.close()




In [ ]:
# Load and preprocess data (from local CSV file)
mydf = pd.read_csv("dep.csv")  
mydf['StatusLabel'] = mydf['Status'].map({0: 'Not Depressed', 1: 'Depressed'})
mydf['SuicideCat'] = mydf['Suicide'].map({0: 'Not Committed', 1: 'Committed Suicide'})
features = ['JobStatus', 'Job', 'SleepDuration', 'DietaryHabits', 'Qualification', 'Suicide', 'Family']
X = pd.get_dummies(mydf[features + ['Gender']], drop_first=False)
ytrue = mydf['Status']
sensitive_feature = mydf['Gender']
feature_names = X.columns.tolist()

In [ ]:
#..... If you have other scripts, you can just initiate from here!

In [ ]:
# Run experiments
for seed_idx, seed in enumerate(seeds):
    for exp in experiments:
        all_metrics_before = []
        all_metrics_after = []
        all_fairness_before = []
        all_fairness_after = []
        all_bias_diagnosis = []
        all_feature_importance = []  # Store feature importance for each flip
        
        # DataFrames to store results for this seed
        metrics_before_df = []
        metrics_after_df = []
        fairness_before_df = []
        fairness_after_df = []
        bias_diagnosis_df = []
        
        for flip_pct in flip_percentages:
            np.random.seed(seed)
            mydf2 = mydf.copy()
            mydf2['statusflip'] = mydf2['StatusLabel'].copy()
            mask = (mydf2['Gender'] == 'Female') & (mydf2['statusflip'] == 'Not Depressed')
            if mask.sum() > 0:
                mydf2.loc[mask, 'statusflip'] = np.random.choice(
                    ['Not Depressed', 'Depressed'],
                    size=mask.sum(),
                    p=[1 - flip_pct, flip_pct]
                )
            mydf2['statusflipbin'] = mydf2['statusflip'].map({'Not Depressed': 0, 'Depressed': 1})
            ybias = mydf2['statusflipbin']
            
            X_temp, X_test, ytrue_temp, ytrue_test, ybias_temp, ybias_test, sensitive_temp, sensitive_test = train_test_split(
                X, ytrue, ybias, sensitive_feature, test_size=0.2, random_state=seed, stratify=ytrue
            )
            X_train, X_val, ytrue_train, ytrue_val, ybias_train, ybias_val, sensitive_train, sensitive_val = train_test_split(
                X_temp, ytrue_temp, ybias_temp, sensitive_temp, test_size=0.25, random_state=seed, stratify=ytrue_temp
            )
            
            y_train = ytrue_train if exp['train'] == 'true' else ybias_train
            y_val_for_mitigation = ytrue_val if exp['train'] == 'true' else ybias_val
            y_test = ytrue_test if exp['test'] == 'true' else ybias_test
            
            ####################################################################
            # Use LGBMClassifier with is_unbalance=True for imbalance
            gbm = LGBMClassifier(random_state=seed, is_unbalance=True)
            gbm.fit(X_train, y_train)
            y_pred = gbm.predict(X_test)
            y_pred_proba = gbm.predict_proba(X_test)
            y_pred_mitigated = apply_mitigation(gbm, X_val, y_pred_proba, sensitive_test, y_val_for_mitigation, sensitive_val)

            ####################################################################
            # Store feature importance for this flip percentage
            importance = gbm.feature_importances_  # Use feature importances from LGBMClassifier
            all_feature_importance.append(importance)
            ####################################################################

            male_mask = (sensitive_test == 'Male')
            female_mask = (sensitive_test == 'Female')
            overall_metrics = compute_metrics(y_test, y_pred, group_name='Overall')
            male_metrics = compute_metrics(y_test[male_mask], y_pred[male_mask], group_name='Male')
            female_metrics = compute_metrics(y_test[female_mask], y_pred[female_mask], group_name='Female')
            fairness = fairness_diagnosis(male_metrics, female_metrics)
            
            overall_metrics_mitigated = compute_metrics(y_test, y_pred_mitigated, group_name='Overall')
            male_metrics_mitigated = compute_metrics(y_test[male_mask], y_pred_mitigated[male_mask], group_name='Male')
            female_metrics_mitigated = compute_metrics(y_test[female_mask], y_pred_mitigated[female_mask], group_name='Female')
            fairness_mitigated = fairness_diagnosis(male_metrics_mitigated, female_metrics_mitigated)
            
            bias_diag = bias_diagnosis(ytrue_test, ybias_test, sensitive_test)
            
            all_metrics_before.append({'Overall': overall_metrics, 'Male': male_metrics, 'Female': female_metrics})
            all_metrics_after.append({'Overall': overall_metrics_mitigated, 'Male': male_metrics_mitigated, 'Female': female_metrics_mitigated})
            all_fairness_before.append(fairness)
            all_fairness_after.append(fairness_mitigated)
            all_bias_diagnosis.append(bias_diag)
            
            # Store results for this seed in DataFrames
            for group in ['Male', 'Female']:
                metrics_before_df.append({
                    'Flip Percentage': flip_pct * 100,
                    'Group': group,
                    **{metric: all_metrics_before[-1][group][metric] for metric in metric_names}
                })
                metrics_after_df.append({
                    'Flip Percentage': flip_pct * 100,
                    'Group': group,
                    **{metric: all_metrics_after[-1][group][metric] for metric in metric_names}
                })
            fairness_before_df.append({
                'Flip Percentage': flip_pct * 100,
                **{row['Metric']: row['Value'] for _, row in fairness.iterrows()}
            })
            fairness_after_df.append({
                'Flip Percentage': flip_pct * 100,
                **{row['Metric']: row['Value'] for _, row in fairness_mitigated.iterrows()}
            })
            bias_diagnosis_df.append({
                'Flip Percentage': flip_pct * 100,
                'Male Error Rate': bias_diag['Male'],
                'Female Error Rate': bias_diag['Female']
            })
        
        all_experiment_results[exp['name']]['metrics_before'][seed_idx] = all_metrics_before
        all_experiment_results[exp['name']]['metrics_after'][seed_idx] = all_metrics_after
        all_experiment_results[exp['name']]['fairness_before'][seed_idx] = all_fairness_before
        all_experiment_results[exp['name']]['fairness_after'][seed_idx] = all_fairness_after
        all_experiment_results[exp['name']]['bias_diagnosis'][seed_idx] = all_bias_diagnosis
        all_experiment_results[exp['name']]['feature_importance'][seed_idx] = all_feature_importance
        
        # Save seed-specific results to CSVs
        pd.DataFrame(metrics_before_df).to_csv(f"metrics_before_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(metrics_after_df).to_csv(f"metrics_after_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(fairness_before_df).to_csv(f"fairness_before_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(fairness_after_df).to_csv(f"fairness_after_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(bias_diagnosis_df).to_csv(f"bias_diagnosis_{exp['name']}_seed_{seed}.csv", index=False)

In [ ]:
# Function to calculate mean and standard error
def calculate_stats(data_list, metric, group=None):
    if group:
        values = np.array([[m[group][metric] for m in metrics] for metrics in data_list])
    else:
        values = np.array([[df[df['Metric'] == metric]['Value'].values[0] for df in dfs] for dfs in data_list])
    mean_values = np.mean(values, axis=0)
    se_values = sem(values, axis=0)
    return mean_values, se_values

# Generate plots, print mean and SE, and save mean/SE CSVs
for exp in experiments:
    exp_name = exp['name']
    results = all_experiment_results[exp_name]
    
    # Plot VIP across seeds for this experiment
    plot_vip_across_seeds(results['feature_importance'], flip_percentages, feature_names, exp_name)
    
    # DataFrames for mean and SE
    metrics_mean_se_df = []
    fairness_mean_se_df = []
    bias_diagnosis_mean_se_df = []
    
    # Print mean and SE for standard metrics and store in DataFrame
    for group in ['Male', 'Female']:
        for metric in metric_names:
            mean_values_before, se_values_before = calculate_stats(results['metrics_before'], metric, group)
            mean_values_after, se_values_after = calculate_stats(results['metrics_after'], metric, group)
            print(f"{exp_name} {group} {metric} - Before Mean: {mean_values_before}, SE: {se_values_before}")
            print(f"{exp_name} {group} {metric} - After Mean: {mean_values_after}, SE: {se_values_after}")
            
            for i, flip_pct in enumerate(flip_percentages):
                metrics_mean_se_df.append({
                    'Flip Percentage': flip_pct * 100,
                    'Group': group,
                    'Metric': metric,
                    'Before Mean': mean_values_before[i],
                    'Before SE': se_values_before[i],
                    'After Mean': mean_values_after[i],
                    'After SE': se_values_after[i]
                })
    
    # Plot metrics (with overall line and mean ± SE annotations)
    plot_metrics_across_flips(results['metrics_before'], [p*100 for p in flip_percentages], metric_names,
                             f"{exp_name} - Before Mitigation", f"metrics_vs_flip_{exp_name}_before.pdf")
    plot_metrics_across_flips(results['metrics_after'], [p*100 for p in flip_percentages], metric_names,
                             f"{exp_name} - After Mitigation", f"metrics_vs_flip_{exp_name}_after.pdf")
    
    # Fairness metrics plot (with mean ± SE annotations)
    plt.figure(figsize=(12, 8))
    for i, metric in enumerate(fairness_metrics):
        before_values = np.array([[df[df['Metric'] == metric]['Value'].values[0] for df in dfs] for dfs in results['fairness_before']])
        after_values = np.array([[df[df['Metric'] == metric]['Value'].values[0] for df in dfs] for dfs in results['fairness_after']])
        before_mean = np.mean(before_values, axis=0)
        after_mean = np.mean(after_values, axis=0)
        before_se = sem(before_values, axis=0)
        after_se = sem(after_values, axis=0)
        
        # Calculate overall mean and SE across all flip percentages
        overall_before_mean = np.mean(before_mean)
        overall_before_se = np.mean(before_se)
        overall_after_mean = np.mean(after_mean)
        overall_after_se = np.mean(after_se)
        
        print(f"{exp_name} Fairness {metric} - Before Mean: {before_mean}, SE: {before_se}")
        print(f"{exp_name} Fairness {metric} - After Mean: {after_mean}, SE: {after_se}")
        
        for j, flip_pct in enumerate(flip_percentages):
            fairness_mean_se_df.append({
                'Flip Percentage': flip_pct * 100,
                'Metric': metric,
                'Before Mean': before_mean[j],
                'Before SE': before_se[j],
                'After Mean': after_mean[j],
                'After SE': after_se[j]
            })
        
        plt.subplot(2, 2, i+1)
        plt.plot([p*100 for p in flip_percentages], before_mean, 'o-', color='#1f77b4', label='Before Mean')
        plt.fill_between([p*100 for p in flip_percentages], before_mean - before_se, before_mean + before_se, color='#1f77b4', alpha=0.2, label='Before Mean ± SE')
        plt.plot([p*100 for p in flip_percentages], after_mean, 'o-', color='#ff7f0e', label='After Mean')
        plt.fill_between([p*100 for p in flip_percentages], after_mean - after_se, after_mean + after_se, color='#ff7f0e', alpha=0.2, label='After Mean ± SE')
        
        # Add overall mean ± SE as text annotation
        plt.text(0.02, 0.98, f'Before: {overall_before_mean:.4f} ± {overall_before_se:.4f}\nAfter: {overall_after_mean:.4f} ± {overall_after_se:.4f}',
                 transform=plt.gca().transAxes, fontsize=8, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.xlabel('Flip Percentage')
        plt.ylabel(metric)
        plt.title(f'{metric} vs Flip Percentage')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.ylim(0.0, 1.0)  # Set y-axis limits to 0.0 to 1.0
    
    plt.suptitle(f'Fairness Metrics vs Flip Percentage ({exp_name})', fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    with PdfPages(f"fairness_vs_flip_{exp_name}.pdf") as pdf:
        pdf.savefig()
    plt.close()
    
    # Bias diagnosis plot (with overall line and mean ± SE annotations)
    plt.figure(figsize=(10, 6))
    male_bias = np.array([[bias['Male'] for bias in biases] for biases in results['bias_diagnosis']])
    female_bias = np.array([[bias['Female'] for bias in biases] for biases in results['bias_diagnosis']])
    male_mean = np.mean(male_bias, axis=0)
    female_mean = np.mean(female_bias, axis=0)
    male_se = sem(male_bias, axis=0)
    female_se = sem(female_bias, axis=0)
    
    # Calculate overall bias (mean of Male and Female error rates)
    overall_bias = (male_bias + female_bias) / 2
    overall_mean = np.mean(overall_bias, axis=0)
    overall_se = sem(overall_bias, axis=0)
    
    # Calculate overall mean and SE across all flip percentages
    overall_male_mean = np.mean(male_mean)
    overall_male_se = np.mean(male_se)
    overall_female_mean = np.mean(female_mean)
    overall_female_se = np.mean(female_se)
    overall_overall_mean = np.mean(overall_mean)
    overall_overall_se = np.mean(overall_se)
    
    print(f"{exp_name} Bias Diagnosis - Male Mean: {male_mean}, SE: {male_se}")
    print(f"{exp_name} Bias Diagnosis - Female Mean: {female_mean}, SE: {female_se}")
    print(f"{exp_name} Bias Diagnosis - Overall Mean: {overall_mean}, SE: {overall_se}")
    
    for i, flip_pct in enumerate(flip_percentages):
        bias_diagnosis_mean_se_df.append({
            'Flip Percentage': flip_pct * 100,
            'Male Mean': male_mean[i],
            'Male SE': male_se[i],
            'Female Mean': female_mean[i],
            'Female SE': female_se[i],
            'Overall Mean': overall_mean[i],
            'Overall SE': overall_se[i]
        })
    
    plt.plot([p*100 for p in flip_percentages], male_mean, 'o-', color='#1f77b4', label='Male Mean')
    plt.fill_between([p*100 for p in flip_percentages], male_mean - male_se, male_mean + male_se, color='#1f77b4', alpha=0.2, label='Male Mean ± SE')
    plt.plot([p*100 for p in flip_percentages], female_mean, 'o-', color='#ff7f0e', label='Female Mean')
    plt.fill_between([p*100 for p in flip_percentages], female_mean - female_se, female_mean + female_se, color='#ff7f0e', alpha=0.2, label='Female Mean ± SE')
    plt.plot([p*100 for p in flip_percentages], overall_mean, 'o-', color='#2ca02c', label='Overall Mean')
    plt.fill_between([p*100 for p in flip_percentages], overall_mean - overall_se, overall_mean + overall_se, color='#2ca02c', alpha=0.2, label='Overall Mean ± SE')
    
    # Add overall mean ± SE as text annotation
    plt.text(0.02, 0.98, f'Male: {overall_male_mean:.4f} ± {overall_male_se:.4f}\nFemale: {overall_female_mean:.4f} ± {overall_female_se:.4f}\nOverall: {overall_overall_mean:.4f} ± {overall_overall_se:.4f}',
             transform=plt.gca().transAxes, fontsize=8, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.xlabel('Flip Percentage')
    plt.ylabel('Bias Error Rate')
    plt.title(f'Bias Error Rate vs Flip Percentage ({exp_name})')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(0.0, 1.0)  # Set y-axis limits to 0.0 to 1.0
    plt.tight_layout()
    with PdfPages(f"bias_error_vs_flip_{exp_name}.pdf") as pdf:
        pdf.savefig()
    plt.close()
    
    # Save mean and SE results to CSVs
    pd.DataFrame(metrics_mean_se_df).to_csv(f"metrics_mean_se_{exp_name}.csv", index=False)
    pd.DataFrame(fairness_mean_se_df).to_csv(f"fairness_mean_se_{exp_name}.csv", index=False)
    pd.DataFrame(bias_diagnosis_mean_se_df).to_csv(f"bias_diagnosis_mean_se_{exp_name}.csv", index=False)